<a href="https://colab.research.google.com/github/poppvk935/Astrophysics-Deep-Learning/blob/main/Tutorial/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import torch

In [52]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [53]:
#Transform the data
#mnist images are 2d but we need a 4d tensor, (number of images, height, width, color channel)
#Convern MNIST image files into tensor of 4 Dimensions
transform = transforms.ToTensor()

In [54]:
#Train data

train_data = datasets.MNIST(root = '/cnn_data', train = True, download = True, transform = transform)

In [55]:
#Test data
test_data = datasets.MNIST(root = '/cnn_data', train = False, download = True, transform = transform)

In [56]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /cnn_data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [57]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /cnn_data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [58]:
#Create a loader for the data
  #need to create batch size, small size
train_loader = DataLoader(train_data, batch_size = 10, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 10, shuffle = False)


In [59]:
#Define CNN Model
#Describe convolutional layer and what its doing (2 conv layers)
#just an example

conv1 = nn.Conv2d(1, 6, 3, 1)
conv2 = nn.Conv2d(6, 16, 3, 1) #6 inputs go with 6 outputs


In [60]:
#Grab 1 MNIST Record/image
for i, (X_train, y_train) in enumerate(train_data):
  break;

In [61]:
X_train.shape

torch.Size([1, 28, 28])

In [62]:
#change to 4d batch
x = X_train.view(1,1,28,28 ) #1 batch of 1 image of 28heightx28width

In [63]:
#perform first convolution
x = F.relu(conv1(x)) #relu is activation function


In [64]:
#1 single image, 6 filters/feature maps, 26x26 because we didn't set padding
x.shape

torch.Size([1, 6, 26, 26])

In [65]:
#pass through pooling layer
x = F.max_pool2d(x, 2,2) #kernel size 2, stride size 2

In [66]:
#26/2 = 13
x.shape

torch.Size([1, 6, 13, 13])

In [67]:
#Do second conv layer

x = F.relu(conv2(x))

In [68]:
x.shape

torch.Size([1, 16, 11, 11])

In [69]:
#pooling layer again

x = F.max_pool2d(x, 2, 2)

In [70]:
x.shape

torch.Size([1, 16, 5, 5])

In [72]:
#Model class
class ConvNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    #fully connected layers
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
  def forward(self,X):
    #First pass
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X, 2,2)
    # Second pass
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X, 2,2)
    #Re-view to flatten it out
    X = X.view(-1, 16*5*5) #negative so we can vary the batch size

    #Fully connected layers
    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X =(self.fc3(X))

    return F.log_softmax(X, dim = 1)

In [73]:
#Create an instance of the Model and set manual seed

torch.manual_seed(41)
model = ConvNN()
model

ConvNN(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [74]:
#Loss Function Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #the smaller the learning rate, the longer it takes to train
